In [ ]:
from openai import OpenAI
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects
import subprocess
import numpy as np

import scienceplots  # for Times New Roman letters

plt.style.use("science")

# Make the front letter

In [ ]:
# Function to add a neon glow effect to text
def neon_glow_text(ax, x, y, text, fontsize=72, color="cyan", glow_radius=60, steps=40):
    # fontsizes = np.linspace(fontsize, fontsize + glow_radius, steps)  # Create a range of font sizes
    for step in range(steps):
        gamma = 7
        alpha = (
            np.sin(step / steps) ** gamma
        )  # Increasing (goes from transparent to opaque)
        r = glow_radius * (
            1 - (step + 1) / steps
        )  # Decreasing (goes from thick to thin)
        text_obj = ax.text(
            x,
            y,
            text,
            fontsize=fontsize,
            ha="center",
            va="center",
            color=color,
            alpha=alpha,
        )
        text_obj.set_path_effects(
            [
                path_effects.withStroke(linewidth=r, foreground=color),
                path_effects.Normal(),
            ]
        )


color = (0, 255, 255)  # cyan
# color = (176, 38, 255) # neon purple
# Convert to [0, 1] range
color = tuple(c / 255 for c in color)

# Center of text and ellipse
x = 0.483
y = 0.5

# Create a figure with a transparent background to be filled in later
fig, ax = plt.subplots(figsize=(10.24, 10.24), dpi=100)  # 1024x1024 pixels
neon_glow_text(ax, x, y, "Score-Models", fontsize=72, color=color)
ax.axis("off")
# Create a black ellipse background that fades to transparent
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect("equal")

# Add rings around it
num_rings = 10  # Number of concentric rings based on radius and width
glow_radius = 0.5  # Radius of the outermost ring
max_alpha = 0.4  # Maximum alpha value for the innermost ring
min_alpha = 0.0  # Minimum alpha value for the outermost ring
ring_width = 0.3
for i in range(num_rings):
    radius = glow_radius * ((i + 1) / num_rings)
    alpha = max_alpha - (max_alpha - min_alpha) * (i / num_rings)
    ring_patch = patches.Wedge(
        (x, y),
        radius,
        0,
        360,
        width=ring_width,
        color="black",
        alpha=alpha,
        transform=ax.transAxes,
    )
    ax.add_patch(ring_patch)

ellipse_patch = patches.Ellipse((x, y), 0.8, 0.2, color="black", alpha=1)
ax.add_patch(ellipse_patch)


plt.savefig("../assets/sbm_mask.png", transparent=True)

# Generate the background based on a prompt of DALL-E 3

In [ ]:
client = OpenAI()

prompt = "The image should depict a cyberpunk/neon cityscape at night with vibrant buildings, \
a clear night sky, a moon, and giant spiral galaxies spread through the top half of the picture, going far out into the Universe."


response = client.images.generate(
    model="dall-e-3",
    prompt=prompt,
    size="1024x1024",
    quality="standard",  # other option is "hd"
    n=1,
)
image_url = response.data[0].url

In [ ]:
# Download the image
subprocess.run(["wget", "-O", "../assets/background.png", image_url])

# Make the composite

In [ ]:
# Open mask and background images
mask = Image.open("../assets/sbm_mask.png").convert("RGBA")
background = Image.open("../assets/background.png").convert("RGBA")

# Resize mask to match background size
mask = mask.resize(background.size, Image.LANCZOS)

# Composite
final_image = Image.alpha_composite(background, mask)

# Save the final image with the glow effect
final_image.save("../assets/logo.png")

In [ ]:
display(final_image)